In [ ]:
!pip install fuzzywuzzy
#pip install python-Levenshtein

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)

In [ ]:
import requests
import pandas as pd

api_key = '30190321fd22f1cbb363d873c33ef154'
base_url = "https://api.themoviedb.org/3"
search_url = f"{base_url}/search/tv?api_key={api_key}&query="

def get_video_url(video):
    if video['site'] == "YouTube":
        return f"https://www.youtube.com/watch?v={video['key']}"
    elif video['site'] == "Vimeo":
        return f"https://vimeo.com/{video['key']}"
    else:
        return None  # Unknown video site

def get_additional_details(show_id):
    videos_url = f"{base_url}/tv/{show_id}/videos?api_key={api_key}"
    videos_resp = requests.get(videos_url).json().get('results', [])
    videos = [get_video_url(video) for video in videos_resp if get_video_url(video)]

    external_ids_url = f"{base_url}/tv/{show_id}/external_ids?api_key={api_key}"
    external_ids_data = requests.get(external_ids_url).json()

    keywords_url = f"{base_url}/tv/{show_id}/keywords?api_key={api_key}"
    keywords_data = requests.get(keywords_url).json().get('results', [])
    keywords = ", ".join(keyword['name'] for keyword in keywords_data)

    return {
        "Videos": ", ".join(videos),
        "IMDb ID": external_ids_data.get("imdb_id", "N/A"),
        "TVDB ID": external_ids_data.get("tvdb_id", "N/A"),
        "Instagram ID": external_ids_data.get("instagram_id", "N/A"),
        "Twitter ID": external_ids_data.get("twitter_id", "N/A"),
        "Keywords": keywords
    }

show_details_list = []

shows = ["The Acolyte","The Boys","The Lord of the Rings: The Rings of Power", "House of the Dragon"]


# Fetch and collect details for each show
for show in shows:
    try:
        search_resp = requests.get(search_url + requests.utils.quote(show))
        search_data = search_resp.json()
        if search_data['results']:
            show_id = search_data['results'][0]['id']  # Taking the first result as the best match
            additional_details = get_additional_details(show_id)
            details_url = f"{base_url}/tv/{show_id}?api_key={api_key}&append_to_response=credits"
            details_resp = requests.get(details_url).json()

            show_info = {
                "ID": show_id,
                "Name": details_resp.get("name"),
                "Overview": details_resp.get("overview"),
                "Popularity": details_resp.get("popularity"),
                "Poster Path": f"https://image.tmdb.org/t/p/original{details_resp.get('poster_path')}",
                "Backdrop Path": f"https://image.tmdb.org/t/p/original{details_resp.get('backdrop_path')}",
                "First Air Date": details_resp.get("first_air_date"),
                "Origin Country": ', '.join(details_resp.get("origin_country", [])),
                "Original Language": details_resp.get("original_language"),
                "Vote Average": details_resp.get("vote_average"),
                "Vote Count": details_resp.get("vote_count"),
                "Status": details_resp.get("status"),
                "Runtime": details_resp.get("episode_run_time")[0] if details_resp.get("episode_run_time") else 'N/A',
                "Genres": ', '.join([genre['name'] for genre in details_resp.get("genres", [])]),
                "Tmdb URL": f"https://www.themoviedb.org/tv/{show_id}",
                "Season": len(details_resp.get("seasons", [])),
                "Networks": ', '.join([network['name'] for network in details_resp.get("networks", [])]),
                "Production Companies": ', '.join([company['name'] for company in details_resp.get("production_companies", [])]),
                "Created By": ', '.join([creator['name'] for creator in details_resp.get("created_by", [])])
            }
            show_info.update(additional_details)
            show_details_list.append(show_info)
        else:
            print(f"Error processing {show}: No results found")
    except Exception as e:
        print(f"Error processing {show}: {e}")

# Convert the list of dictionaries to a DataFrame
shows_df = pd.DataFrame(show_details_list)

In [ ]:
shows_df.shape

(4, 25)

In [ ]:
shows_df.head()

,ID,Name,Overview,Popularity,Poster Path,Backdrop Path,First Air Date,Origin Country,Original Language,Vote Average,Vote Count,Status,Runtime,Genres,Tmdb URL,Season,Networks,Production Companies,Created By,Videos,IMDb ID,TVDB ID,Instagram ID,Twitter ID,Keywords
0,114479,The Acolyte,"An investigation into a shocking crime spree pits a respected Jedi Master against a dangerous warrior from his past. As more clues emerge, they travel down a dark path where sinister forces reveal all is not what it seems.",970.100,https://image.tmdb.org/t/p/original/mztdt3y6GBsJR69zHtszFezTCLT.jpg,https://image.tmdb.org/t/p/original/kwronSXO1ogMqHHFvY2eBxfFLdn.jpg,2024-06-04,US,en,5.614,176,Returning Series,42,"Mystery, Sci-Fi & Fantasy",https://www.themoviedb.org/tv/114479,1,Disney+,"Lucasfilm Ltd., Shoot to Midnight",Leslye Headland,"https://www.youtube.com/watch?v=Hl7HG4PbZWE, https://www.youtube.com/watch?v=7Dju63U24m0, https://www.youtube.com/watch?v=6poFQjvrsco, https://www.youtube.com/watch?v=lIF1B7S6ogY, https://www.youtube.com/watch?v=9DItelHk3ng, https://www.youtube.com/watch?v=9NRL4pCcZ8Q, https://www.youtube.com/watch?v=RXF0IfbgpXQ, https://www.youtube.com/watch?v=Wb_zdHIQOj8, https://www.youtube.com/watch?v=0DaWPcYe2pc, https://www.youtube.com/watch?v=zotV8szyesw, https://www.youtube.com/watch?v=QIrF3elBNS8, https://www.youtube.com/watch?v=ABX8upKvrto, https://www.youtube.com/watch?v=B0WxdRadrm8, https://www.youtube.com/watch?v=c3hCVk5Z9po, https://www.youtube.com/watch?v=Py_IYpy4IlQ, https://www.youtube.com/watch?v=7BCP4JSMoKc, https://www.youtube.com/watch?v=6tzur6JrUEA, https://www.youtube.com/watch?v=BtytYWhg2mc",tt12262202,393197,officialacolyte,OfficialAcolyte,"monk, witch, magic, twins, lgbt, murder investigation, based on movie, crime investigation, mysterious, lgbt parenting, spaceship, mystery"
1,76479,The Boys,A group of vigilantes known informally as “The Boys” set out to take down corrupt superheroes with no more than blue-collar grit and a willingness to fight dirty.,3436.959,https://image.tmdb.org/t/p/original/2zmTngn1tYC1AvfnrFLhxeD82hz.jpg,https://image.tmdb.org/t/p/original/7cqKGQMnNabzOpi7qaIgZvQ7NGV.jpg,2019-07-25,US,en,8.471,9618,Returning Series,60,"Sci-Fi & Fantasy, Action & Adventure",https://www.themoviedb.org/tv/76479,6,Prime Video,"Amazon Studios, Original Film, Sony Pictures Television Studios, Kripke Enterprises, Point Grey Pictures, Kickstart Entertainment, NightSky Productions, Amazon MGM Studios",Eric Kripke,"https://www.youtube.com/watch?v=EzFXDvC-EwM, https://www.youtube.com/watch?v=tcrNsIaQkb4, https://www.youtube.com/watch?v=PnWEyGnC4cg, https://www.youtube.com/watch?v=bo28a58xFJE, https://www.youtube.com/watch?v=uR766FtEhv0, https://www.youtube.com/watch?v=CD46c08MsHg, https://www.youtube.com/watch?v=NilteC-7jeM, https://www.youtube.com/watch?v=FG1EByNnHUU",tt1190634,355567,theboystv,theboystv,"superhero, based on comic, revenge, gore, superhero team, sign languages"
2,84773,The Lord of the Rings: The Rings of Power,"Beginning in a time of relative peace, we follow an ensemble cast of characters as they confront the re-emergence of evil to Middle-earth. From the darkest depths of the Misty Mountains, to the majestic forests of Lindon, to the breathtaking island kingdom of Númenor, to the furthest reaches of the map, these kingdoms and characters will carve out legacies that live on long after they are gone.",211.227,https://image.tmdb.org/t/p/original/mYLOqiStMxDK3fYZFirgrMt8z5d.jpg,https://image.tmdb.org/t/p/original/qBppESpY8e97WfPWVZiU0JdRXw.jpg,2022-09-01,US,en,7.342,2498,Returning Series,N/A,"Action & Adventure, Sci-Fi & Fantasy, Drama",https://www.themoviedb.org/tv/84773,3,Prime Video,"Amazon Studios, New Line Cinema","John D. Payne, Patrick McKay","https://www.youtube.com/watch?v=IqW3rw5UNUo, https://www.youtube.com/watch?v=x8UAUAuKNcU, https://www.youtube.com/watch?v=xaSZNPscvFU, https://www.youtube.com/watch?v=uYnQDsaxHZU, https://www.youtube.com/watch?v=ewgCqJDI_Nk, https://www.youtube.co

In [ ]:
def get_show_id(api_key, show_name, base_url="https://api.themoviedb.org/3"):
    search_url = f"{base_url}/search/tv?api_key={api_key}&query={requests.utils.quote(show_name)}"
    response = requests.get(search_url)
    results = response.json().get('results', [])
    if not results:
        return None, None
    show_id = results[0]['id']
    return show_id, results[0].get('name', 'Unknown')

def fetch_imdb_id(api_key, show_id, base_url="https://api.themoviedb.org/3"):
    """Fetch IMDb ID for the show."""
    external_ids_url = f"{base_url}/tv/{show_id}/external_ids?api_key={api_key}"
    response = requests.get(external_ids_url)
    ids_data = response.json()
    return ids_data.get('imdb_id')

def fetch_cast_and_crew_details(api_key, show_id, base_url="https://api.themoviedb.org/3"):
    """Fetch cast and crew details for a show."""
    credits_url = f"{base_url}/tv/{show_id}/aggregate_credits?api_key={api_key}"
    response = requests.get(credits_url)
    credits_data = response.json()

    # Cast details
    cast = credits_data.get('cast', [])
    cast_summary = {actor['name']: f"{actor.get('roles', [{}])[0].get('character', 'Unknown Character')} ({actor.get('roles', [{}])[0].get('episode_count', 0)} Episodes)"
                    for actor in cast}

    # Crew details
    crew = credits_data.get('crew', [])
    crew_summary = {}
    for member in crew:
        for job in member.get('jobs', []):
            name = member['name']
            job_description = f"{job['job']} ({job['episode_count']} Episodes)"
            if name in crew_summary:
                crew_summary[name].add(job_description)
            else:
                crew_summary[name] = {job_description}
    for key in crew_summary:
        crew_summary[key] = "; ".join(sorted(crew_summary[key]))

    return cast_summary, len(cast), crew_summary, len(crew)

def summarize_show_credits(api_key, show_names):
    """Create a DataFrame summarizing credit details for multiple shows."""
    base_url = "https://api.themoviedb.org/3"
    data = []

    for show_name in show_names:
        show_id, actual_show_name = get_show_id(api_key, show_name, base_url)
        if show_id:
            imdb_id = fetch_imdb_id(api_key, show_id, base_url)
            cast_summary, cast_count, crew_summary, crew_count = fetch_cast_and_crew_details(api_key, show_id, base_url)
            data.append({
                'Show': actual_show_name,
                'IMDb ID': imdb_id,
                'Total Cast': cast_count,
                'Cast Details': cast_summary,
                'Total Crew': crew_count,
                'Crew Details': crew_summary
            })
        else:
            data.append({
                'Show': show_name,
                'IMDb ID': 'No data',
                'Total Cast': 'No data',
                'Cast Details': 'No data',
                'Total Crew': 'No data',
                'Crew Details': 'No data'
            })

    return pd.DataFrame(data)

# Fetch and display the data
api_key = '30190321fd22f1cbb363d873c33ef154'
shows = ["The Acolyte","The Boys","The Lord of the Rings: The Rings of Power", "House of the Dragon"]

df1 = summarize_show_credits(api_key, shows)
#df_cast_crew_tmdb.to_csv("tv_shows_cast_and_crew_details.csv", index=False)


In [ ]:
df1.shape

(4, 6)

In [ ]:
df1.head()

Show     IMDb ID  Total Cast                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
final_df = pd.merge(shows_df, df1, on='IMDb ID', how='inner', suffixes=('_orig', '_cast'))

In [ ]:
final_df

ID                                       Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Overview  Popularity                                                          Poster Path                                                        Backdrop Path First Air Date Origin Country Original Language  Vote Average  Vote Count            Status Runtime                                       Genres                              Tmdb URL  Season     Networks                                                                                                                                                         Production Companies                        Created By                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Videos     IMDb ID  TVDB ID         Instagram ID       Twitter ID                                                                                                                                                                                     Keywords                                       Show  Total Cast                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
final_df.to_csv("final_df.csv", index=False)

In [ ]:
import requests
import pandas as pd

imdb_ids = ["tt12262202","tt1190634", "tt7631058", "tt11198330"]

api_key = 'b6685a2a92msh5fdca4a0268f66bp1b65cfjsna0e8d09a728b'
headers = {
    'X-RapidAPI-Key': api_key,
    'X-RapidAPI-Host': 'online-movie-database.p.rapidapi.com'
}

def get_filming_locations(imdb_id):
    url = f"https://imdb8.p.rapidapi.com/title/get-filming-locations?tconst={imdb_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_company_credits(imdb_id):
    url = f"https://imdb8.p.rapidapi.com/title/get-company-credits?tconst={imdb_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

data = []
for imdb_id in imdb_ids:
    locations = get_filming_locations(imdb_id)
    company_credits = get_company_credits(imdb_id)
    data.append({
        "IMDb ID": imdb_id,
        "Filming Locations": locations,
        "Company Credits": company_credits
    })

df1 = pd.DataFrame(data)


In [ ]:
df1

,IMDb ID,Filming Locations,Company Credits
0,tt12262202,"{'@type': 'imdb.api.title.filminglocations', 'base': {'id': '/title/tt12262202/', 'image': {'height': 2048, 'id': '/title/tt12262202/images/rm4134108929', 'url': 'https://m.media-amazon.com/images/M/MV5BY2RlMmMyNzktNmE1Zi00MWMzLWFkMGEtMzU5OWFhMGMzMzhkXkEyXkFqcGdeQXVyMTQzNTA5MzYz._V1_.jpg', 'width': 1383}, 'runningTimeInMinutes': 35, 'nextEpisode': '/title/tt13820376/', 'numberOfEpisodes': 8, 'seriesStartYear': 2024, 'title': 'The Acolyte', 'titleType': 'tvSeries', 'year': 2024}, 'id': '/title/tt12262202/', 'locations': [{'id': '/title/tt12262202/filminglocations/lc3343524', 'interestingVotes': {'up': 17}, 'location': 'Madeira, Portugal'}, {'extras': ['on location'], 'id': '/title/tt12262202/filminglocations/lc3822535', 'interestingVotes': {'up': 4}, 'location': 'Brecon Beacons National Park, Wales, UK'}, {'id': '/title/tt12262202/filminglocations/lc2085958', 'interestingVotes': {'up': 2}, 'location': 'London, England, UK'}, {'id': '/title/tt12262202/filminglocations/lc3822534', 'interestingVotes': {'up': 2}, 'location': 'Shinfield Studios, Berkshire, UK'}]}","{'categories': [{'id': 'production', 'name': 'Production Companies', 'section': {'items': [{'id': 'co0071326', 'rowTitle': 'Lucasfilm', 'listContent': [{'text': ''}]}, {'id': 'co0721120', 'rowTitle': 'Disney+', 'listContent': [{'text': ''}]}, {'id': 'co0044374', 'rowTitle': 'The Walt Disney Company', 'listContent': [{'text': ''}]}]}}, {'id': 'distribution', 'name': 'Distributors', 'section': {'items': [{'id': 'co0721120', 'rowTitle': 'Disney+', 'listContent': [{'text': '(World-wide, 2024)', 'subText': '(VOD, video)'}]}]}}, {'id': 'specialEffects', 'name': 'Special Effects', 'section': {'items': [{'id': 'co0461574', 'rowTitle': 'Clear Angle Studios', 'listContent': [{'text': ''}]}, {'id': 'co0581035', 'rowTitle': 'Hybride', 'listContent': [{'text': ''}]}, {'id': 'co0072491', 'rowTitle': 'Industrial Light & Magic (ILM)', 'listContent': [{'text': ''}]}, {'id': 'co0116745', 'rowTitle': 'Luma Pictures', 'listContent': [{'text': ''}]}, {'id': 'co0463696', 'rowTitle': 'Outpost VFX', 'listContent': [{'text': ''}]}]}}, {'id': 'miscellaneous', 'name': 'Other Companies', 'section': {'items': [{'id': 'co1054192', 'rowTitle': 'Shinfield Studios', 'listContent': [{'text': '', 'subText': '(studio)'}]}, {'id': 'co0515604', 'rowTitle': 'ARRI Rental', 'listContent': [{'text': '', 'subText': '(camera and grip equipment provided by)'}]}, {'id': 'co0356750', 'rowTitle': 'Above the Line Set Assistance & Security', 'listContent': [{'text': '', 'subText': '(Security)'}]}, {'id': 'co0029274', 'rowTitle': 'Audiolink Radio Communications', 'listContent': [{'text': '', 'subText': '(cell phone rentals)'}]}, {'id': 'co0029274', 'rowTitle': 'Audiolink Radio Communications', 'listContent': [{'text': '', 'subText': '(walkie talkies)'}]}]}}]}"
1,tt1190634,"{'@type': 'imdb.api.title.filminglocations', 'base': {'id': '/title/tt1190634/', 'image': {'height': 1350, 'id': '/title/tt1190634/images/rm4040589057', 'url': 'https://m.media-amazon.com/images/M/MV5BYTY2ZjYyNGUtZGVkZS00MDNhLWIwMjMtZDk4MmQ5ZWI0NTY4XkEyXkFqcGdeQXVyMTY3MDE5MDY1._V1_.jpg', 'width': 1080}, 'runningTimeInMinutes': 60, 'nextEpisode': '/title/tt7775902/', 'numberOfEpisodes': 33, 'seriesStartYear': 2019, 'title': 'The Boys', 'titleType': 'tvSeries', 'year': 2019}, 'id': '/title/tt1190634/', 'locations': [{'extras': ['location'], 'id': '/title/tt1190634/filminglocations/lc1569749', 'interestingVotes': {'down': 2, 'up': 74}, 'location': 'Hamilton, Ontario, Canada'}, {'id': '/title/tt1190634/filminglocations/lc1698110', 'interestingVotes': {'down': 2, 'up': 54}, 'location': 'Toronto, Ontario, Canada'}, {'extras': ['Vought Headquarters'], 'id': '/title/tt1190634/filminglocations/lc1811270', 'interestingVotes': {'down': 1, 'up': 40}, 'location': 'Roy Thomson Hall, Toronto, Ontario, Canada'}, {'extras': ['Hugh and Annie conversation'], 'id': '/title/tt1190634/filminglocations/lc1811271', 'inte

In [ ]:
import pandas as pd

def parse_locations(loc_dict):
    locations = [loc['location'] for loc in loc_dict['locations']]
    return ', '.join(locations)

def parse_company_credits(credits_dict):
    result = []
    for category in credits_dict['categories']:
        for item in category['section']['items']:
            company_info = f"{item['rowTitle']} ({category['name']})"
            if 'listContent' in item and item['listContent']:
                for content in item['listContent']:
                    text = content.get('text', '').strip()
                    sub_text = content.get('subText', '').strip()
                    if text:
                        company_info += f", {text}"
                    if sub_text:
                        company_info += f", {sub_text}"
            result.append(company_info)
    return result

df1['Filming Locations'] = df1['Filming Locations'].apply(parse_locations)
df1['Company Credits'] = df1['Company Credits'].apply(parse_company_credits)


In [ ]:
df1

,IMDb ID,Filming Locations,Company Credits
0,tt12262202,"Madeira, Portugal, Brecon Beacons National Park, Wales, UK, London, England, UK, Shinfield Studios, Berkshire, UK","[Lucasfilm (Production Companies), Disney+ (Production Companies), The Walt Disney Company (Production Companies), Disney+ (Distributors), (World-wide, 2024), (VOD, video), Clear Angle Studios (Special Effects), Hybride (Special Effects), Industrial Light & Magic (ILM) (Special Effects), Luma Pictures (Special Effects), Outpost VFX (Special Effects), Shinfield Studios (Other Companies), (studio), ARRI Rental (Other Companies), (camera and grip equipment provided by), Above the Line Set Assistance & Security (Other Companies), (Security), Audiolink Radio Communications (Other Companies), (cell phone rentals), Audiolink Radio Communications (Other Companies), (walkie talkies)]"
1,tt1190634,"Hamilton, Ontario, Canada, Toronto, Ontario, Canada, Roy Thomson Hall, Toronto, Ontario, Canada, University of Toronto, Toronto, Ontario, Canada, Cinespace Film Studios - 11030 Highway 27, Kleinburg, Ontario, Canada","[Amazon Studios (Production Companies), Kickstart Entertainment (Production Companies), Kripke Enterprises (Production Companies), NightSky Productions (Production Companies), Original Film (Production Companies), Amazon Studios (Distributors), (World-wide, 2019), Amazon Prime Video (Distributors), (United States, 2019), (VOD, video), Amazon Prime Video (Distributors), (Argentina, 2019), (VoD, video), Amazon Prime Video (Distributors), (Australia, 2019), (VoD, video), Amazon Prime Video (Distributors), (Austria, 2019), (VoD, video), AB VFX Studios (Special Effects), (visual effects), BOT VFX (Special Effects), (additional visual effects), Double Negative (DNEG) (Special Effects), (visual effects), Folks (Special Effects), (Visual effects), Folks (Special Effects), (visual effects), Aero Mock-Ups (Other Companies), (aircraft mock-up rental), CNCPT (Other Companies), (previsualization), Company 3 (Other Companies), (post-production facilities), Dynamite Entertainment Comics (Other Companies), (based on the comic book by), Entertainment Partners Canada (Other Companies), (payroll services)]"
2,tt7631058,"Auckland, New Zealand, New Zealand, Auckland Film Studios, New Zealand, Bovingdon Airfield, Chesham Road, Bovingdon, Hemel Hempstead, Hertfordshire, UK, Bray Film Studios, Windsor Road, Water Oakley, Windsor, Berkshire, England, UK, UK, Bovingdon Airfield Studios, Bovingdon Airfield, Chesham Road, Bovingdon, Hemel Hempstead, Hertfordshire, UK","[Amazon Studios (Production Companies), Cause and FX (Production Companies), (Visual Effects), Harper Collins Publishers (Production Companies), New Line Cinema (Production Companies), Reunion Pacific Entertainment (Production Companies), (production services by), Amazon Prime Video (Distributors), (United States, 2022), (VOD, video), Amazon Prime Video (Distributors), (World-wide, 2022), (VOD, video), Amazon Studios (Distributors), Embracer Group (Distributors), (World-wide), PixStone Images (Special Effects), (Visual effects), Double Negative (DNEG) (Special Effects), (visual effects), Industrial Light & Magic (ILM) (Special Effects), Outpost VFX (Special Effects), (visual effects), Weta Workshop (Special Effects), (Concept Design, Prosthetic Make-up Effects and Weapons), AIR Studios (Other Companies), (music recorded at), Abbey Road Studios (Other Companies), (music recorded at), Ags Film & Tv (Other Companies), (transport supplier), Audiolink Radio Communications (Other Companies), (cell phone rentals), Audiolink Radio Communications (Other Companies), (walkie talkies)]"
3,tt11198330,"Monsanto, Castelo Branco, Portugal, Croatia, St Michael's Mount, Cornwall, England, UK, Cáceres, Cáceres, Extremadura, Spain, La Calahorra, Granada, Andalucía, Spain, Plaza de Santa María, Cáceres, Cáceres, Extremadura, Spain, Trujillo, Cáceres, Extremadura, Spain, Castleton, Derbyshire, England, UK, Castillo de La Calahorra, La Calahorr

In [ ]:
import requests
import pandas as pd

imdb_ids = ["tt12262202","tt1190634", "tt7631058", "tt11198330"]

api_key = 'b6685a2a92msh5fdca4a0268f66bp1b65cfjsna0e8d09a728b'
headers = {
    'X-RapidAPI-Key': api_key,
    'X-RapidAPI-Host': 'online-movie-database.p.rapidapi.com'
}

def fetch_keywords(imdb_id):
    url = f"https://imdb8.p.rapidapi.com/title/v2/get-keywords?tconst={imdb_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return {}

def fetch_taglines(imdb_id):
    url = f"https://imdb8.p.rapidapi.com/title/v2/get-taglines?tconst={imdb_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return {}

def parse_keywords(data):
    keywords_list = []
    if 'data' in data and 'title' in data['data'] and 'keywordItemCategories' in data['data']['title']:
        categories = data['data']['title']['keywordItemCategories']
        for category in categories:
            if 'keywords' in category and 'edges' in category['keywords']:
                for edge in category['keywords']['edges']:
                    if 'node' in edge and 'keyword' in edge['node'] and 'text' in edge['node']['keyword']:
                        keywords_list.append(edge['node']['keyword']['text']['text'])
    return keywords_list

def parse_taglines(data):
    taglines_list = []
    if 'data' in data and 'title' in data['data'] and 'taglines' in data['data']['title']:
        if 'edges' in data['data']['title']['taglines']:
            taglines_list = [edge['node']['text'] for edge in data['data']['title']['taglines']['edges']]
    return taglines_list

data = []
for imdb_id in imdb_ids:
    keyword_data = fetch_keywords(imdb_id)
    keywords = parse_keywords(keyword_data)
    tagline_data = fetch_taglines(imdb_id)
    taglines = parse_taglines(tagline_data)
    data.append({
        "IMDb ID": imdb_id,
        "Keywords": ", ".join(keywords),
        "Taglines": ", ".join(taglines)
    })

df2 = pd.DataFrame(data)
print(df2)


      IMDb ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Keywords                                                             Taglines
0  tt12262202                                                                                                                                                                                                                                          

In [ ]:
import requests
import pandas as pd

imdb_ids = ["tt12262202","tt1190634", "tt7631058", "tt11198330"]

api_key = 'b6685a2a92msh5fdca4a0268f66bp1b65cfjsna0e8d09a728b'
headers = {
    'X-RapidAPI-Key': api_key,
    'X-RapidAPI-Host': 'online-movie-database.p.rapidapi.com'
}

def fetch_awards(imdb_id):
    url = f"https://imdb8.p.rapidapi.com/title/v2/get-awards?tconst={imdb_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return {}

def parse_awards(data):
    awards_list = []
    total_awards = 0
    if 'data' in data and 'title' in data['data'] and 'awardNominations' in data['data']['title']:
        nominations = data['data']['title']['awardNominations']
        total_awards = nominations['total']
        if 'edges' in nominations:
            for edge in nominations['edges']:
                if 'node' in edge:
                    node = edge['node']
                    awards_list.append({
                        "Category": node['category']['text'],
                        "Is Winner": node['isWinner'],
                        "Award Name": node['award']['awardName'] if 'award' in node and 'awardName' in node['award'] else 'Unknown',
                        "Event": node['award']['eventEdition']['event']['text'] if 'eventEdition' in node['award'] and 'event' in node['award']['eventEdition'] else 'Unknown',
                        "Year": node['award']['eventEdition']['year'] if 'eventEdition' in node['award'] else 'Unknown'
                    })
    return awards_list, total_awards

data = []
for imdb_id in imdb_ids:
    awards_data = fetch_awards(imdb_id)
    parsed_awards, total_awards = parse_awards(awards_data)
    data.append({
        "IMDb ID": imdb_id,
        "Awards Details": parsed_awards,
        "Total Awards": total_awards
    })

df = pd.DataFrame(data)
df.explode('Awards Details').reset_index(drop=True)
print(df.to_string())


      IMDb ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
def simplify_awards(awards_list):
    summary = []
    for award in awards_list:
        result = "Won" if award['Is Winner'] else "Nominated"
        summary.append(f"{result} {award['Category']} at {award['Event']} ({award['Year']})")
    return ". ".join(summary)

df['Awards Summary'] = df['Awards Details'].apply(simplify_awards)

final_df1 = df[['IMDb ID', 'Awards Summary', 'Total Awards']]

print(final_df1.to_string(index=False))

   IMDb ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
import requests
import pandas as pd

imdb_ids = ["tt12262202", "tt1190634", "tt7631058", "tt11198330"]

genres_url = 'https://imdb8.p.rapidapi.com/title/v2/get-genres'

headers = {
    'X-RapidAPI-Key': 'b6685a2a92msh5fdca4a0268f66bp1b65cfjsna0e8d09a728b',
    'X-RapidAPI-Host': 'online-movie-database.p.rapidapi.com'
}

def get_genres(imdb_id):
    querystring = {"tconst": imdb_id}
    response = requests.get(genres_url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data for {imdb_id}. Status code: {response.status_code}")
        return None

all_data = []
for imdb_id in imdb_ids:
    print(f"Fetching data for IMDb ID: {imdb_id}")
    genres_data = get_genres(imdb_id)

    print(genres_data)

    genres_list = []
    if genres_data and 'data' in genres_data and 'title' in genres_data['data'] and 'titleGenres' in genres_data['data']['title']:
        genres = genres_data['data']['title']['titleGenres']['genres']
        genres_list = [genre['genre']['text'] for genre in genres]

    entry = {
        "IMDb ID": imdb_id,
        "Genres": ", ".join(genres_list)
    }
    all_data.append(entry)

df3= pd.DataFrame(all_data)



Fetching data for IMDb ID: tt12262202
{'data': {'title': {'id': 'tt12262202', 'titleGenres': {'__typename': 'TitleGenres', 'genres': [{'genre': {'genreId': 'Action', 'text': 'Action'}}, {'genre': {'genreId': 'Adventure', 'text': 'Adventure'}}, {'genre': {'genreId': 'Drama', 'text': 'Drama'}}, {'genre': {'genreId': 'Fantasy', 'text': 'Fantasy'}}, {'genre': {'genreId': 'Mystery', 'text': 'Mystery'}}, {'genre': {'genreId': 'Sci-Fi', 'text': 'Sci-Fi'}}, {'genre': {'genreId': 'Thriller', 'text': 'Thriller'}}]}, 'titleType': {'__typename': 'TitleType', 'id': 'tvSeries', 'text': 'TV Series', 'categories': [{'id': 'tv', 'text': 'TV', 'value': 'tv'}], 'canHaveEpisodes': True, 'isEpisode': False, 'isSeries': True, 'displayableProperty': {'value': {'plainText': 'TV Series'}}}}}}
Fetching data for IMDb ID: tt1190634
{'data': {'title': {'id': 'tt1190634', 'titleGenres': {'__typename': 'TitleGenres', 'genres': [{'genre': {'genreId': 'Action', 'text': 'Action'}}, {'genre': {'genreId': 'Comedy', 'text

In [ ]:
df3

,IMDb ID,Genres
0,tt12262202,"Action, Adventure, Drama, Fantasy, Mystery, Sci-Fi, Thriller"
1,tt1190634,"Action, Comedy, Crime, Drama, Sci-Fi"
2,tt7631058,"Action, Adventure, Drama, Fantasy"
3,tt11198330,"Action, Adventure, Drama, Fantasy, Romance"


In [ ]:
#USER REVIEW SUMMARY,CRITICS REVIEW SUMMARY, officialLinks FACEBOOK, INSTAGRAM LINK, PRINCIPAL CREDITS

In [ ]:
result = df1.merge(df2, on='IMDb ID', how='inner').merge(final_df1, on='IMDb ID', how='inner').merge(df3, on='IMDb ID', how='inner')

In [ ]:
result

,IMDb ID,Filming Locations,Company Credits,Keywords,Taglines,Awards Summary,Total Awards,Genres
0,tt12262202,"Madeira, Portugal, Brecon Beacons National Park, Wales, UK, London, England, UK, Shinfield Studios, Berkshire, UK","[Lucasfilm (Production Companies), Disney+ (Production Companies), The Walt Disney Company (Production Companies), Disney+ (Distributors), (World-wide, 2024), (VOD, video), Clear Angle Studios (Special Effects), Hybride (Special Effects), Industrial Light & Magic (ILM) (Special Effects), Luma Pictures (Special Effects), Outpost VFX (Special Effects), Shinfield Studios (Other Companies), (studio), ARRI Rental (Other Companies), (camera and grip equipment provided by), Above the Line Set Assistance & Security (Other Companies), (Security), Audiolink Radio Communications (Other Companies), (cell phone rentals), Audiolink Radio Communications (Other Companies), (walkie talkies)]","space sci fi, space adventure, dark fantasy, star wars, galaxy, lightsaber, fight, republic, dark, master, exploration, outer space, disney star wars, cultural diversity, lgbt character, hero, villain, weapon, light, laser, army, control, space, controversy",In an age of light a darkness rises.,,0,"Action, Adventure, Drama, Fantasy, Mystery, Sci-Fi, Thriller"
1,tt1190634,"Hamilton, Ontario, Canada, Toronto, Ontario, Canada, Roy Thomson Hall, Toronto, Ontario, Canada, University of Toronto, Toronto, Ontario, Canada, Cinespace Film Studios - 11030 Highway 27, Kleinburg, Ontario, Canada","[Amazon Studios (Production Companies), Kickstart Entertainment (Production Companies), Kripke Enterprises (Production Companies), NightSky Productions (Production Companies), Original Film (Production Companies), Amazon Studios (Distributors), (World-wide, 2019), Amazon Prime Video (Distributors), (United States, 2019), (VOD, video), Amazon Prime Video (Distributors), (Argentina, 2019), (VoD, video), Amazon Prime Video (Distributors), (Australia, 2019), (VoD, video), Amazon Prime Video (Distributors), (Austria, 2019), (VoD, video), AB VFX Studios (Special Effects), (visual effects), BOT VFX (Special Effects), (additional visual effects), Double Negative (DNEG) (Special Effects), (visual effects), Folks (Special Effects), (Visual effects), Folks (Special Effects), (visual effects), Aero Mock-Ups (Other Companies), (aircraft mock-up rental), CNCPT (Other Companies), (previsualization), Company 3 (Other Companies), (post-production facilities), Dynamite Entertainment Comics (Other Companies), (based on the comic book by), Entertainment Partners Canada (Other Companies), (payroll services)]","superhero action, superhero sci fi, satire comedy, dark comedy, 2020s, 2010s, violence, blood, enhancement drug, corporate marketing, superhero, vigilante, supernatural power, gore, female nudity, black comedy, female full frontal nudity, superhero versus superhero, ensemble cast, revenge, vengeance, super speed, frenchman abroad, frenchman in new york, new york city, name calling, new zealand actor playing american character, sex scene, based on comic book, two word title, live action comic adaptation, comics literature on screen, live action adaptation, lesbian character, lesbian interest, lgbt, gay, homosexuality, homosexual, gay sex, graphic male genitalia",Never Meet Your Heroes,"Won Best Action (TV Spot / Trailer /Teaser for a Series) at Golden Trailer Awards (2019). Nominated Best Sound Editing (TV Spot/Trailer/Teaser for a Series) at Golden Trailer Awards (2019). Nominated One Hour Contemporary Single-Camera Television Series at Art Directors Guild (2020). Nominated Best TV Comic Book Adaptation at IGN Summer Movie Awards (2019). Nominated Best New TV Series at IGN Summer Movie Awards (2019). Nominated Best TV Ensemble at IGN Summer Movie Awards (2019). Nominated Outstanding Sound Editing for a Comedy or Drama Series (One Hour) at Primetime Emmy Awards (2020). Nominated Best Featured Actor in a TV Series 10-20 Years at The Joey Awards, Vancouver (2020). Won 

In [ ]:
#result.to_csv("result.csv", index=False)

In [ ]:
def direct_categorize_credits(credits_list):
    distributors = []
    production_companies = []
    special_effects = []
    other_companies = []

    for credit in credits_list:
        category, details = credit.split(' (', 1)
        if "Distributors" in details:
            distributors.append(credit)
        elif "Production Companies" in details:
            production_companies.append(credit)
        elif "Special Effects" in details:
            special_effects.append(credit)
        elif "Other Companies" in details:
            other_companies.append(credit)

    return pd.Series([distributors, production_companies, special_effects, other_companies])

result[['Distributors', 'Production Companies', 'Special Effects', 'Other Companies']] = result['Company Credits'].apply(direct_categorize_credits)

result[['Distributors', 'Production Companies', 'Special Effects', 'Other Companies']].head()


,Distributors,Production Companies,Special Effects,Other Companies
0,"[Disney+ (Distributors), (World-wide, 2024), (VOD, video)]","[Lucasfilm (Production Companies), Disney+ (Production Companies), The Walt Disney Company (Production Companies)]","[Clear Angle Studios (Special Effects), Hybride (Special Effects), Industrial Light & Magic (ILM) (Special Effects), Luma Pictures (Special Effects), Outpost VFX (Special Effects)]","[Shinfield Studios (Other Companies), (studio), ARRI Rental (Other Companies), (camera and grip equipment provided by), Above the Line Set Assistance & Security (Other Companies), (Security), Audiolink Radio Communications (Other Companies), (cell phone rentals), Audiolink Radio Communications (Other Companies), (walkie talkies)]"
1,"[Amazon Studios (Distributors), (World-wide, 2019), Amazon Prime Video (Distributors), (United States, 2019), (VOD, video), Amazon Prime Video (Distributors), (Argentina, 2019), (VoD, video), Amazon Prime Video (Distributors), (Australia, 2019), (VoD, video), Amazon Prime Video (Distributors), (Austria, 2019), (VoD, video)]","[Amazon Studios (Production Companies), Kickstart Entertainment (Production Companies), Kripke Enterprises (Production Companies), NightSky Productions (Production Companies), Original Film (Production Companies)]","[AB VFX Studios (Special Effects), (visual effects), BOT VFX (Special Effects), (additional visual effects), Double Negative (DNEG) (Special Effects), (visual effects), Folks (Special Effects), (Visual effects), Folks (Special Effects), (visual effects)]","[Aero Mock-Ups (Other Companies), (aircraft mock-up rental), CNCPT (Other Companies), (previsualization), Company 3 (Other Companies), (post-production facilities), Dynamite Entertainment Comics (Other Companies), (based on the comic book by), Entertainment Partners Canada (Other Companies), (payroll services)]"
2,"[Amazon Prime Video (Distributors), (United States, 2022), (VOD, video), Amazon Prime Video (Distributors), (World-wide, 2022), (VOD, video), Amazon Studios (Distributors), Embracer Group (Distributors), (World-wide)]","[Amazon Studios (Production Companies), Cause and FX (Production Companies), (Visual Effects), Harper Collins Publishers (Production Companies), New Line Cinema (Production Companies), Reunion Pacific Entertainment (Production Companies), (production services by)]","[PixStone Images (Special Effects), (Visual effects), Double Negative (DNEG) (Special Effects), (visual effects), Industrial Light & Magic (ILM) (Special Effects), Outpost VFX (Special Effects), (visual effects), Weta Workshop (Special Effects), (Concept Design, Prosthetic Make-up Effects and Weapons)]","[AIR Studios (Other Companies), (music recorded at), Abbey Road Studios (Other Companies), (music recorded at), Ags Film & Tv (Other Companies), (transport supplier), Audiolink Radio Communications (Other Companies), (cell phone rentals), Audiolink Radio Communications (Other Companies), (walkie talkies)]"
3,"[24TV (Distributors), (Russia, 2023), (TV), Amediateka (Distributors), (Russia, 2022), (TV), HBO Latin America (Distributors), (Argentina), (TV), HBO Max (Distributors), (United States, 2022), (VOD, video), Home Box Office (HBO) (Distributors), (United States, 2022), (TV)]","[1:26 Pictures (Production Companies), Home Box Office (HBO) (Production Companies)]","[MPC Episodic (Special Effects), NVIZ (Special Effects), (visualisation), Outpost VFX (Special Effects), (visual effects), Pixomondo (Special Effects), Red Visual Effects (Special Effects), (visual effects)]","[ARRI Rental (Other Companies), (camera and grip equipment provided by), Aluzine Rentals (Other Companies), (grip and lighting equipment), Casting Collective (Other Companies), (extras casting), Czech Film Choir (Other Companies), (music performed by), Czech Film Orchestra (Other Companies), (music performed by)]"


In [ ]:
def list_to_string(entries):
    return ';'.join(entries)

result['Distributors'] = result['Distributors'].apply(list_to_string)
result['Production Companies'] = result['Production Companies'].apply(list_to_string)
result['Special Effects'] = result['Special Effects'].apply(list_to_string)
result['Other Companies'] = result['Other Companies'].apply(list_to_string)

result[['Distributors', 'Production Companies', 'Special Effects', 'Other Companies']].head()


,Distributors,Production Companies,Special Effects,Other Companies
0,"Disney+ (Distributors), (World-wide, 2024), (VOD, video)",Lucasfilm (Production Companies);Disney+ (Production Companies);The Walt Disney Company (Production Companies),Clear Angle Studios (Special Effects);Hybride (Special Effects);Industrial Light & Magic (ILM) (Special Effects);Luma Pictures (Special Effects);Outpost VFX (Special Effects),"Shinfield Studios (Other Companies), (studio);ARRI Rental (Other Companies), (camera and grip equipment provided by);Above the Line Set Assistance & Security (Other Companies), (Security);Audiolink Radio Communications (Other Companies), (cell phone rentals);Audiolink Radio Communications (Other Companies), (walkie talkies)"
1,"Amazon Studios (Distributors), (World-wide, 2019);Amazon Prime Video (Distributors), (United States, 2019), (VOD, video);Amazon Prime Video (Distributors), (Argentina, 2019), (VoD, video);Amazon Prime Video (Distributors), (Australia, 2019), (VoD, video);Amazon Prime Video (Distributors), (Austria, 2019), (VoD, video)",Amazon Studios (Production Companies);Kickstart Entertainment (Production Companies);Kripke Enterprises (Production Companies);NightSky Productions (Production Companies);Original Film (Production Companies),"AB VFX Studios (Special Effects), (visual effects);BOT VFX (Special Effects), (additional visual effects);Double Negative (DNEG) (Special Effects), (visual effects);Folks (Special Effects), (Visual effects);Folks (Special Effects), (visual effects)","Aero Mock-Ups (Other Companies), (aircraft mock-up rental);CNCPT (Other Companies), (previsualization);Company 3 (Other Companies), (post-production facilities);Dynamite Entertainment Comics (Other Companies), (based on the comic book by);Entertainment Partners Canada (Other Companies), (payroll services)"
2,"Amazon Prime Video (Distributors), (United States, 2022), (VOD, video);Amazon Prime Video (Distributors), (World-wide, 2022), (VOD, video);Amazon Studios (Distributors);Embracer Group (Distributors), (World-wide)","Amazon Studios (Production Companies);Cause and FX (Production Companies), (Visual Effects);Harper Collins Publishers (Production Companies);New Line Cinema (Production Companies);Reunion Pacific Entertainment (Production Companies), (production services by)","PixStone Images (Special Effects), (Visual effects);Double Negative (DNEG) (Special Effects), (visual effects);Industrial Light & Magic (ILM) (Special Effects);Outpost VFX (Special Effects), (visual effects);Weta Workshop (Special Effects), (Concept Design, Prosthetic Make-up Effects and Weapons)","AIR Studios (Other Companies), (music recorded at);Abbey Road Studios (Other Companies), (music recorded at);Ags Film & Tv (Other Companies), (transport supplier);Audiolink Radio Communications (Other Companies), (cell phone rentals);Audiolink Radio Communications (Other Companies), (walkie talkies)"
3,"24TV (Distributors), (Russia, 2023), (TV);Amediateka (Distributors), (Russia, 2022), (TV);HBO Latin America (Distributors), (Argentina), (TV);HBO Max (Distributors), (United States, 2022), (VOD, video);Home Box Office (HBO) (Distributors), (United States, 2022), (TV)",1:26 Pictures (Production Companies);Home Box Office (HBO) (Production Companies),"MPC Episodic (Special Effects);NVIZ (Special Effects), (visualisation);Outpost VFX (Special Effects), (visual effects);Pixomondo (Special Effects);Red Visual Effects (Special Effects), (visual effects)","ARRI Rental (Other Companies), (camera and grip equipment provided by);Aluzine Rentals (Other Companies), (grip and lighting equipment);Casting Collective (Other Companies), (extras casting);Czech Film Choir (Other Companies), (music performed by);Czech Film Orchestra (Other Companies), (music performed by)"


In [ ]:
def remove_duplicates_from_locations(location_str):
    unique_locations = set([loc.strip() for loc in location_str.split(',')])
    return ', '.join(sorted(unique_locations))

result['Filming Locations'] = result['Filming Locations'].apply(remove_duplicates_from_locations)

result[['Filming Locations']].head()

,Filming Locations
0,"Berkshire, Brecon Beacons National Park, England, London, Madeira, Portugal, Shinfield Studios, UK, Wales"
1,"Canada, Cinespace Film Studios - 11030 Highway 27, Hamilton, Kleinburg, Ontario, Roy Thomson Hall, Toronto, University of Toronto"
2,"Auckland, Auckland Film Studios, Berkshire, Bovingdon, Bovingdon Airfield, Bovingdon Airfield Studios, Bray Film Studios, Chesham Road, England, Hemel Hempstead, Hertfordshire, New Zealand, UK, Water Oakley, Windsor, Windsor Road"
3,"Aldershot, Andalucía, Caesar's Camp, California, Castelo Branco, Castillo de La Calahorra, Castleton, Catalonia, Cornwall, Croatia, Cáceres, Derbyshire, England, Extremadura, Girona, Granada, Hampshire, Hertfordshire, Holywell Bay, La Calahorra, Leavesden, Lloret de Mar, Los Angeles, Monsanto, Newquay, Plaza de Santa María, Portugal, Spain, St Michael's Mount, The Santa Clotilde Gardens, Trujillo, UK, USA, Warner Bros. Studios Leavesden, Warner Drive, Watford"


In [ ]:
def categorize_awards(awards_summary):
    awards_won = []
    awards_nominated = []
    awards_entries = awards_summary.split('. ')
    for entry in awards_entries:
        if entry.startswith('Won'):
            awards_won.append(entry)
        elif entry.startswith('Nominated'):
            awards_nominated.append(entry)

    return '; '.join(awards_won), '; '.join(awards_nominated)

result[['Awards Won', 'Awards Nominated']] = result['Awards Summary'].apply(lambda x: pd.Series(categorize_awards(x)))

result[['Awards Won', 'Awards Nominated']].head()


,Awards Won,Awards Nominated
0,,
1,Won Best Action (TV Spot / Trailer /Teaser for a Series) at Golden Trailer Awards (2019); Won Best Superhero Series at Critics Choice Super Awards (2021); Won Best Actor in a Superhero Series at Critics Choice Super Awards (2021); Won Best Actress in a Superhero Series at Critics Choice Super Awards (2021); Won Best Villain in a Series at Critics Choice Super Awards (2021); Won Best Dramatic TV Performance at IGN Summer Movie Awards (2020),"Nominated Best Sound Editing (TV Spot/Trailer/Teaser for a Series) at Golden Trailer Awards (2019); Nominated One Hour Contemporary Single-Camera Television Series at Art Directors Guild (2020); Nominated Best TV Comic Book Adaptation at IGN Summer Movie Awards (2019); Nominated Best New TV Series at IGN Summer Movie Awards (2019); Nominated Best TV Ensemble at IGN Summer Movie Awards (2019); Nominated Outstanding Sound Editing for a Comedy or Drama Series (One Hour) at Primetime Emmy Awards (2020); Nominated Best Featured Actor in a TV Series 10-20 Years at The Joey Awards, Vancouver (2020); Nominated Best Actor in a Superhero Series at Critics Choice Super Awards (2021); Nominated Outstanding Achievement in Casting - Television Pilot and First Season - Drama at Casting Society of America, USA (2021); Nominated Best Television Series, Comedy or Musical at Satellite Awards (2021); Nominated Outstanding Action Performance by a Stunt Ensemble in a Comedy or Drama Series at Screen Actors Guild Awards (2021); Nominated Best TV Ensemble at IGN Summer Movie Awards (2020); Nominated Drama Series at Writers Guild of America, USA (2021); Nominated Excellence in Crafts - Textile Arts at The CAFTCAD awards (2021)"
2,"Won Best Theme for a TV Series at The Movie Music International - Recognition Awards (2023); Won Outstanding Visual Effects in a Photoreal Episode at Visual Effects Society Awards (2023); Won Outstanding Created Environment in an Episode, Commercial, or Real-Time Project at Visual Effects Society Awards (2023); Won Outstanding Effects Simulations in an Episode, Commercial, or Real-Time Project at Visual Effects Society Awards (2023); Won Score of the Year at Movie Music UK Awards (2023); Won Best Original Score for Television at Movie Music UK Awards (2023); Won Best Cinematography at LifeArt Festival (2022); Won Score of the Year at International Film Music Critics Award (IFMCA) (2023)","Nominated TV Series Competition at Camerimage (2022); Nominated The Sci-Fi/Fantasy Show of 2022 at People's Choice Awards, USA (2022); Nominated Best Original Score - TV Show/Limited Series at Hollywood Music In Media Awards (HMMA) (2022); Nominated Best Supporting Actor in a Drama Series at Critics Choice Awards (2023); Nominated Outstanding Original Score for a Television Production at Society of Composers and Lyricists Awards (2023); Nominated Outstanding Achievement in Sound Editing - Broadcast Long Form Dialogue/ADR at Motion Picture Sound Editors, USA (2023); Nominated Outstanding Action Performance by a Stunt Ensemble in a Television Series at Screen Actors Guild Awards (2023); Nominated Excellence in Sci-Fi/Fantasy Television at Costume Designers Guild Awards (2023); Nominated Outstanding Created Environment in an Episode, Commercial, or Real-Time Project at Visual Effects Society Awards (2023); Nominated Outstanding Effects Simulations in an Episode, Commercial, or Real-Time Project at Visual Effects Society Awards (2023); Nominated Outstanding Compositing and Lighting in an Episode at Visual Effects Society Awards (2023); Nominated Outstanding Special (Practical) Effects in a Photoreal Project at Visual Effects Society Awards (2023)"
3,"Won Best Television Series - Drama at Golden Globes, USA (2023); Won Excellence in Sci-Fi/Fantasy Television at Costume Designers Guild Awards (2023); Won Best TV Series at Golden Tomato Awards (Rotten Tomatoes) (2022); Won Best New Series at Golden Tomato Awards (Rotten Tomatoes) (2022); Won Best Sci-Fi & Fantasy Series at Gol

In [ ]:
import re
def parse_and_categorize_awards(awards_summary):
    pattern = re.compile(r"(Won|Nominated) (.+?) at ([^(]+) \((\d{4})\)")
    awards_data = []

    matches = pattern.findall(awards_summary)
    for result, category, award_name, year in matches:
        awards_data.append({
            "Result": result,
            "Category": category,
            "Award Name": award_name.strip(),
            "Year": year
        })

    return awards_data

result['Parsed Awards'] = result['Awards Summary'].apply(parse_and_categorize_awards)

In [ ]:
result.to_csv("result.csv", index=False)

NameError: name 'result' is not defined

In [ ]:
result

,IMDb ID,Filming Locations,Company Credits,Keywords,Taglines,Awards Summary,Total Awards,Genres,Distributors,Production Companies,Special Effects,Other Companies,Awards Won,Awards Nominated,Parsed Awards
0,tt12262202,"Berkshire, Brecon Beacons National Park, England, London, Madeira, Portugal, Shinfield Studios, UK, Wales","[Lucasfilm (Production Companies), Disney+ (Production Companies), The Walt Disney Company (Production Companies), Disney+ (Distributors), (World-wide, 2024), (VOD, video), Clear Angle Studios (Special Effects), Hybride (Special Effects), Industrial Light & Magic (ILM) (Special Effects), Luma Pictures (Special Effects), Outpost VFX (Special Effects), Shinfield Studios (Other Companies), (studio), ARRI Rental (Other Companies), (camera and grip equipment provided by), Above the Line Set Assistance & Security (Other Companies), (Security), Audiolink Radio Communications (Other Companies), (cell phone rentals), Audiolink Radio Communications (Other Companies), (walkie talkies)]","space sci fi, space adventure, dark fantasy, star wars, galaxy, lightsaber, fight, republic, dark, master, exploration, outer space, disney star wars, cultural diversity, lgbt character, hero, villain, weapon, light, laser, army, control, space, controversy",In an age of light a darkness rises.,,0,"Action, Adventure, Drama, Fantasy, Mystery, Sci-Fi, Thriller","Disney+ (Distributors), (World-wide, 2024), (VOD, video)",Lucasfilm (Production Companies);Disney+ (Production Companies);The Walt Disney Company (Production Companies),Clear Angle Studios (Special Effects);Hybride (Special Effects);Industrial Light & Magic (ILM) (Special Effects);Luma Pictures (Special Effects);Outpost VFX (Special Effects),"Shinfield Studios (Other Companies), (studio);ARRI Rental (Other Companies), (camera and grip equipment provided by);Above the Line Set Assistance & Security (Other Companies), (Security);Audiolink Radio Communications (Other Companies), (cell phone rentals);Audiolink Radio Communications (Other Companies), (walkie talkies)",,,[]
1,tt1190634,"Canada, Cinespace Film Studios - 11030 Highway 27, Hamilton, Kleinburg, Ontario, Roy Thomson Hall, Toronto, University of Toronto","[Amazon Studios (Production Companies), Kickstart Entertainment (Production Companies), Kripke Enterprises (Production Companies), NightSky Productions (Production Companies), Original Film (Production Companies), Amazon Studios (Distributors), (World-wide, 2019), Amazon Prime Video (Distributors), (United States, 2019), (VOD, video), Amazon Prime Video (Distributors), (Argentina, 2019), (VoD, video), Amazon Prime Video (Distributors), (Australia, 2019), (VoD, video), Amazon Prime Video (Distributors), (Austria, 2019), (VoD, video), AB VFX Studios (Special Effects), (visual effects), BOT VFX (Special Effects), (additional visual effects), Double Negative (DNEG) (Special Effects), (visual effects), Folks (Special Effects), (Visual effects), Folks (Special Effects), (visual effects), Aero Mock-Ups (Other Companies), (aircraft mock-up rental), CNCPT (Other Companies), (previsualization), Company 3 (Other Companies), (post-production facilities), Dynamite Entertainment Comics (Other Companies), (based on the comic book by), Entertainment Partners Canada (Other Companies), (payroll services)]","superhero action, superhero sci fi, satire comedy, dark comedy, 2020s, 2010s, violence, blood, enhancement drug, corporate marketing, superhero, vigilante, supernatural power, gore, female nudity, black comedy, female full frontal nudity, superhero versus superhero, ensemble cast, revenge, vengeance, super speed, frenchman abroad, frenchman in new york, new york city, name calling, new zealand actor playing american character, sex scene, based on comic book, two word title, live action comic adaptation, comics literature on screen, live action adaptation, lesbian character, lesbian interest, lgbt, gay, homosexuality, homosexual, gay sex, graphic male genitalia",Never Meet Your Heroes,"Wo

In [ ]:
final_df

ID                                       Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Overview  Popularity                                                          Poster Path                                                        Backdrop Path First Air Date Origin Country Original Language  Vote Average  Vote Count            Status Runtime                                       Genres                              Tmdb URL  Season     Networks                                                                                                                                                         Production Companies                        Created By                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Videos     IMDb ID  TVDB ID         Instagram ID       Twitter ID                                                                                                                                                                                     Keywords                                       Show  Total Cast                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
merged_df = pd.merge(result, final_df, on='IMDb ID', how='inner')

In [ ]:
merged_df

IMDb ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                Filming Locations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Company Credits                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Keywords_x                                                             Taglines                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
merged_df.to_csv("merged_df.csv", index=False)

In [ ]:
30190321fd22f1cbb363d873c33ef154

SyntaxError: invalid decimal literal (<ipython-input-37-39d07e7aefb5>, line 1)

In [ ]:
import requests
import pandas as pd

def get_show_id(api_key, show_name, base_url="https://api.themoviedb.org/3"):
    search_url = f"{base_url}/search/tv?api_key={api_key}&query={requests.utils.quote(show_name)}"
    try:
        response = requests.get(search_url)
        response.raise_for_status()
        results = response.json().get('results', [])
        if not results:
            return None, "No results found"
        show_id = results[0]['id']
        return show_id, results[0].get('name', 'Unknown')
    except requests.RequestException as e:
        return None, f"Error fetching data: {str(e)}"

def fetch_imdb_id(api_key, show_id, base_url="https://api.themoviedb.org/3"):
    external_ids_url = f"{base_url}/tv/{show_id}/external_ids?api_key={api_key}"
    try:
        response = requests.get(external_ids_url)
        response.raise_for_status()
        ids_data = response.json()
        return ids_data.get('imdb_id')
    except requests.RequestException as e:
        return f"Error fetching IMDb ID: {str(e)}"

def fetch_cast_and_crew_details(api_key, show_id, base_url="https://api.themoviedb.org/3"):
    credits_url = f"{base_url}/tv/{show_id}/aggregate_credits?api_key={api_key}"
    try:
        response = requests.get(credits_url)
        response.raise_for_status()
        credits_data = response.json()

        cast = credits_data.get('cast', [])
        cast_summary = "; ".join([f"{actor['name']} as {actor.get('roles', [{}])[0].get('character', 'Unknown')} ({actor.get('roles', [{}])[0].get('episode_count', 0)} eps)"
                                  for actor in cast])

        crew = credits_data.get('crew', [])
        crew_summary = "; ".join([f"{member['name']} ({', '.join([job['job'] for job in member.get('jobs', [])])})"
                                  for member in crew])

        return cast_summary, len(cast), crew_summary, len(crew)
    except requests.RequestException as e:
        return "Error fetching credits", 0, "Error fetching credits", 0

def summarize_show_credits(api_key, show_names):
    base_url = "https://api.themoviedb.org/3"
    data = []

    for show_name in show_names:
        show_id, actual_show_name = get_show_id(api_key, show_name, base_url)
        if show_id:
            imdb_id = fetch_imdb_id(api_key, show_id, base_url)
            cast_summary, cast_count, crew_summary, crew_count = fetch_cast_and_crew_details(api_key, show_id, base_url)
            data.append({
                'Show': actual_show_name,
                'IMDb ID': imdb_id,
                'Total Cast': cast_count,
                'Cast Details': cast_summary
            })
        else:
            data.append({
                'Show': show_name,
                'IMDb ID': actual_show_name,
                'Total Cast': 0,
                'Cast Details': 'No data available'
            })

    return pd.DataFrame(data)

api_key = '30190321fd22f1cbb363d873c33ef154'
shows = ["The Acolyte", "The Boys", "The Lord of the Rings: The Rings of Power", "House of the Dragon"]

df = summarize_show_credits(api_key, shows)
df


In [ ]:
30190321fd22f1cbb363d873c33ef154

In [ ]:
import requests
import pandas as pd

def get_show_id(api_key, show_name, base_url="https://api.themoviedb.org/3"):
    """Retrieve the TMDB show ID based on show name."""
    search_url = f"{base_url}/search/tv?api_key={api_key}&query={requests.utils.quote(show_name)}"
    try:
        response = requests.get(search_url)
        response.raise_for_status()
        results = response.json().get('results', [])
        if not results:
            return None, "No results found"
        return results[0]['id'], results[0].get('name', 'Unknown')
    except requests.RequestException as e:
        return None, f"Error fetching data: {str(e)}"

def fetch_imdb_id(api_key, show_id, base_url="https://api.themoviedb.org/3"):
    """Fetch IMDb ID for the show."""
    external_ids_url = f"{base_url}/tv/{show_id}/external_ids?api_key={api_key}"
    try:
        response = requests.get(external_ids_url)
        response.raise_for_status()
        ids_data = response.json()
        return ids_data.get('imdb_id')
    except requests.RequestExcepltion as e:
        return f"Error fetching IMDb ID: {str(e)}"

def fetch_cast_and_crew_details(api_name, show_id, base_url="https://api.themoviedb.org/3"):
    """Fetch cast and crew details for a show, structured by department."""
    credits_url = f"{base_url}/tv/{show_id}/aggregate_credits?api_key={api_key}"
    try:
        response = requests.get(credits_url)
        response.raise_for_status()
        credits_data = response.json()

        cast = credits_data.get('cast', [])
        cast_summary = "; ".join([f"{actor['name']} as {actor.get('roles', [{}])[0].get('character', 'Unknown')} ({actor.get('roles', [{}])[0].get('episode_count', 0)} eps)"
                                  for actor in cast])


        crew = credits_data.get('crew', [])
        department_details = {}
        for member in crew:
            department = member.get('department', 'Other')
            if member.get('jobs'):
                for job in member['jobs']:
                    job_description = f"{member['name']} - {job['job']} ({job['episode_count']} eps)"
                    if department in department_details:
                        department_details[department].append(job_description)
                    else:
                        department_details[department] = [job_description]

        return cast_summary, len(cast), department_details, len(crew)
    except requests.RequestException as e:
        return "Error fetching credits", 0, {}, 0

def summarize_show_credits(api_key, show_names):
    """Create a DataFrame summarizing credit details for multiple shows."""
    base_url = "https://api.themoviedb.org/3"
    data = []

    for show_name in show_names:
        show_id, actual_show_name = get_show_id(api_key, show_name, base_url)
        if show_id:
            imdb_id = fetch_imdb_id(api_key, show_id, base_url)
            cast_summary, cast_count, crew_details, crew_count = fetch_cast_and_crew_details(api_key, show_id, base_url)
            show_data = {
                'Show': actual_show_name,
                'IMDb ID': imdb_id,
                'Total Cast': cast_count,
                'Cast Details': cast_summary,
                'Total Crew': crew_count,
            }
            show_data.update({f"Crew - {dept}": "; ".join(details) for dept, details in crew_details.items()})
            data.append(show_data)
        else:
            data.append({
                'Show': show_name,
                'IMDb ID': actual_show_name,
                'Total Cast': 0,
                'Cast Details': 'No data available',
                'Total Crew': 0,
            })

    return pd.DataFrame(data)

api_key = '30190321fd22f1cbb363d873c33ef154'
shows = ["The Acolyte", "The Boys", "The Lord of the Rings: The Rings of Power", "House of the Dragon"]

df = summarize_show_credits(api_key, shows)
df